In [2]:
import pandas as pd
from datasets import load_dataset

data = load_dataset("csv", data_files="../input/real-and-synthetic-megablunders/megablunders.csv")

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# Define label list based on your grammar error categories
label_list = ["AGREE", "DM", "MM", "ROS", "PR", "PAR", "FRAG", "CASE", "NONE"]
label_to_id = {label: i for i, label in enumerate(label_list)}

# Function to convert string labels to numerical indices
def convert_labels(examples):
    # Make sure the column name matches your dataset's label column
    # If your label column is not named "label", change it accordingly
    examples["label"] = [label_to_id[label] for label in examples["label"]]
    return examples

# Apply the conversion to your dataset
data = data.map(convert_labels, batched=True)

# Verify the conversion worked
print(f"Converted labels: {data['train']['label'][:5]}")

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Converted labels: [1, 6, 4, 0, 5]


In [6]:
data['train'][:5]

{'text': ['By dropping a game to the pathetic Tampa Bay Devil Rays, we may be on the verge of another Red Sox collapse.',
  'Although, if history is any indication, the tease could still go on for several weeks.',
  'The Sox almost never go down uneventfully, which is the most frustrating thing for their legion of fans.',
  'Because of the accumulated bad karma that hangs over the franchise, the skill of the players in any given year seem to be irrelevant to the outcome.',
  'The team not only has squandered huge leads but also it has come roaring from behind only to fall short.to fall short.'],
 'label': [1, 6, 4, 0, 5]}

In [7]:
def tokenize(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

data = data.map(tokenize, batched=True)

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=9)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

2025-05-23 23:27:36.007078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748042856.198234      70 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748042856.252518      70 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, 

In [9]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [18]:
import numpy as np
import evaluate 

metric = evaluate.load("accuracy")

def compute_metrics(eval_prod):
    loigts, labels = eval_prod
    
    predictions = np.argmax(loigts, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

trainingArguments = TrainingArguments(
    output_dir="megablunders",
    eval_strategy="epoch", 
    push_to_hub=False, 
    report_to="none",

)

In [19]:
from transformers import Trainer


from transformers import Trainer
from datasets import DatasetDict

# Split the training data into train and test sets
train_test_split = data["train"].train_test_split(test_size=0.2, seed=42)
data = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

trainer = Trainer(
    model=model,
    args=trainingArguments,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.001624,0.375000
2,No log,1.895678,0.500000
3,No log,1.857319,0.541667


TrainOutput(global_step=36, training_loss=1.859574211968316, metrics={'train_runtime': 27.0774, 'train_samples_per_second': 10.415, 'train_steps_per_second': 1.33, 'total_flos': 74201980925952.0, 'train_loss': 1.859574211968316, 'epoch': 3.0})